In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 50ms/step - loss: 10.9801 - accuracy: 0.4863
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 6.6651 - accuracy: 0.5246
Epoch 3/50
8/8 [==============================] - 0s 47ms/step - loss: 4.9996 - accuracy: 0.5608
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 4.3070 - accuracy: 0.5805
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 3.7850 - accuracy: 0.6331
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 3.3611 - accuracy: 0.7054
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 2.9779 - accuracy: 0.8357
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 2.6614 - accuracy: 0.8949
Epoch 9/50
8/8 [==============================] - 0s 48ms/step - loss: 2.4321 - accuracy: 0.9277
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 2.3050 - accuracy: 0.9277
Epoch 11/50
8/8 [===========

8/8 [==============================] - 0s 49ms/step - loss: 1.0484 - accuracy: 0.9967
Epoch 35/50
8/8 [==============================] - 0s 49ms/step - loss: 1.0364 - accuracy: 0.9901
Epoch 36/50
8/8 [==============================] - 0s 47ms/step - loss: 1.0067 - accuracy: 0.9967
Epoch 37/50
8/8 [==============================] - 0s 48ms/step - loss: 0.9801 - accuracy: 0.9967
Epoch 38/50
8/8 [==============================] - 0s 48ms/step - loss: 0.9504 - accuracy: 0.9978
Epoch 39/50
8/8 [==============================] - 0s 48ms/step - loss: 0.9205 - accuracy: 1.0000
Epoch 40/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8996 - accuracy: 0.9967
Epoch 41/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8892 - accuracy: 0.9956
Epoch 42/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8715 - accuracy: 0.9967
Epoch 43/50
8/8 [==============================] - 0s 48ms/step - loss: 0.8455 - accuracy: 1.0000
Epoch 44/50
8/8 [===============

8/8 [==============================] - 0s 49ms/step - loss: 1.6377 - accuracy: 0.9595
Epoch 18/50
8/8 [==============================] - 0s 49ms/step - loss: 1.5425 - accuracy: 0.9715
Epoch 19/50
8/8 [==============================] - 0s 50ms/step - loss: 1.4433 - accuracy: 0.9781
Epoch 20/50
8/8 [==============================] - 0s 48ms/step - loss: 1.3813 - accuracy: 0.9748
Epoch 21/50
8/8 [==============================] - 0s 49ms/step - loss: 1.3468 - accuracy: 0.9759
Epoch 22/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3374 - accuracy: 0.9759
Epoch 23/50
8/8 [==============================] - 0s 49ms/step - loss: 1.3026 - accuracy: 0.9704
Epoch 24/50
8/8 [==============================] - 0s 48ms/step - loss: 1.2499 - accuracy: 0.9825
Epoch 25/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1914 - accuracy: 0.9847
Epoch 26/50
8/8 [==============================] - 0s 48ms/step - loss: 1.1672 - accuracy: 0.9792
Epoch 27/50
8/8 [===============

8/8 [==============================] - 0s 48ms/step - loss: 0.7204 - accuracy: 0.9934
 >#5: 96.943 
[99.12663698196411, 98.68995547294617, 98.25327396392822, 99.56331849098206, 96.94322943687439]
 Accuracy: 98.515% (+/-0.899) 


In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),input_shape=(None,n_length,n_features))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 11.0673 - accuracy: 0.5126
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 6.7860 - accuracy: 0.6079
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 4.9999 - accuracy: 0.6725
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 4.2194 - accuracy: 0.8478
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 3.6594 - accuracy: 0.8850
Epoch 6/50
8/8 [==============================] - 0s 51ms/step - loss: 3.2314 - accuracy: 0.9430
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 2.9512 - accuracy: 0.9507
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 2.7397 - accuracy: 0.9639
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 2.6805 - accuracy: 0.9398
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 2.5079 - accuracy: 0.9518
Epoch 11/50
8/8 [===========

Epoch 27/50
8/8 [==============================] - 0s 49ms/step - loss: 1.3697 - accuracy: 0.9704
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2863 - accuracy: 0.9858
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 1.2106 - accuracy: 0.9901
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1341 - accuracy: 0.9956
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0815 - accuracy: 0.9945
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0395 - accuracy: 0.9956
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0041 - accuracy: 0.9989
Epoch 34/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9691 - accuracy: 1.0000
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9394 - accuracy: 0.9989
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9173 - accuracy: 0.9956
Epoch 37/50
8/8 [===

 >#3: 96.943 
Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 11.0401 - accuracy: 0.5071
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 6.7199 - accuracy: 0.5367
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 5.0071 - accuracy: 0.6090
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 4.2877 - accuracy: 0.6780
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 3.8096 - accuracy: 0.7010
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 3.3330 - accuracy: 0.8576
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 3.0452 - accuracy: 0.8729
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 2.7466 - accuracy: 0.9354
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 2.5502 - accuracy: 0.9354
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 2.4386 - accuracy: 0.9211
Epoch 11/50
8/

8/8 [==============================] - 0s 51ms/step - loss: 1.1203 - accuracy: 0.9978
Epoch 27/50
8/8 [==============================] - 0s 51ms/step - loss: 1.1730 - accuracy: 0.9639
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1362 - accuracy: 0.9836
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1290 - accuracy: 0.9814
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0689 - accuracy: 0.9923
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0264 - accuracy: 0.9945
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9943 - accuracy: 0.9989
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9795 - accuracy: 0.9912
Epoch 34/50
8/8 [==============================] - 0s 49ms/step - loss: 0.9438 - accuracy: 0.9978
Epoch 35/50
8/8 [==============================] - 0s 49ms/step - loss: 0.9181 - accuracy: 0.9945
Epoch 36/50
8/8 [===============

In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 12.8753 - accuracy: 0.5312
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 10.4236 - accuracy: 0.5827
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 8.2475 - accuracy: 0.6145
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 6.6349 - accuracy: 0.7196
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 5.5214 - accuracy: 0.7831
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 4.7141 - accuracy: 0.8598
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 4.2111 - accuracy: 0.8970
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 3.8510 - accuracy: 0.9277
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 3.5898 - accuracy: 0.9332
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 3.3694 - accuracy: 0.9671
Epoch 11/50
8/8 [==========

8/8 [==============================] - 0s 51ms/step - loss: 1.6669 - accuracy: 0.9912
Epoch 27/50
8/8 [==============================] - 0s 49ms/step - loss: 1.5826 - accuracy: 0.9989
Epoch 28/50
8/8 [==============================] - 0s 52ms/step - loss: 1.5082 - accuracy: 0.9978
Epoch 29/50
8/8 [==============================] - 0s 49ms/step - loss: 1.4473 - accuracy: 0.9923
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 1.4824 - accuracy: 0.9858
Epoch 31/50
8/8 [==============================] - 0s 51ms/step - loss: 1.6034 - accuracy: 0.9781
Epoch 32/50
8/8 [==============================] - 0s 52ms/step - loss: 1.5952 - accuracy: 0.9945
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 1.5228 - accuracy: 0.9912
Epoch 34/50
8/8 [==============================] - 0s 52ms/step - loss: 1.4058 - accuracy: 0.9956
Epoch 35/50
8/8 [==============================] - 0s 52ms/step - loss: 1.2925 - accuracy: 0.9978
Epoch 36/50
8/8 [===============

 >#3: 55.895 
Epoch 1/50
8/8 [==============================] - 0s 53ms/step - loss: 12.4051 - accuracy: 0.5290
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 9.4839 - accuracy: 0.6199
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 7.2790 - accuracy: 0.7393
Epoch 4/50
8/8 [==============================] - 0s 51ms/step - loss: 5.9155 - accuracy: 0.8149
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 5.0546 - accuracy: 0.8686
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 4.4737 - accuracy: 0.9124
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 4.0358 - accuracy: 0.9595
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 3.7273 - accuracy: 0.9660
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 3.4093 - accuracy: 0.9814
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 3.1513 - accuracy: 0.9869
Epoch 11/50
8/

8/8 [==============================] - 0s 51ms/step - loss: 1.5462 - accuracy: 0.9989
Epoch 27/50
8/8 [==============================] - 0s 51ms/step - loss: 1.4587 - accuracy: 0.9956
Epoch 28/50
8/8 [==============================] - 0s 52ms/step - loss: 1.4603 - accuracy: 0.9803
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 1.5024 - accuracy: 0.9912
Epoch 30/50
8/8 [==============================] - 0s 51ms/step - loss: 1.4732 - accuracy: 0.9945
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3672 - accuracy: 0.9967
Epoch 32/50
8/8 [==============================] - 0s 53ms/step - loss: 1.2680 - accuracy: 0.9989
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 1.1878 - accuracy: 0.9978
Epoch 34/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1195 - accuracy: 1.0000
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0654 - accuracy: 1.0000
Epoch 36/50
8/8 [===============